### First, import the necessary modules, get the webpage and extract the data frames from tables using Pandas

In [18]:
import requests
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
torontodf = df_list[0]
torontodf.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Rename columns

In [19]:
torontodf.columns = torontodf.iloc[0,]
torontodf.drop(0,0,inplace=True)
torontodf.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Extract to a different data frame only the rows which have an assigned borough

In [21]:
torontodf2 = torontodf.loc[torontodf['Borough'] != "Not assigned",:]
torontodf2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [22]:
torontodf2.shape

(212, 3)

### Group rows with the same Postcode and concatenate the Neighbourhood separated by comma

In [30]:
torontodf3 = torontodf2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
torontodf2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### For any unassigned neighbourhood, replace with the value of the borough

In [38]:
for i in range(torontodf3.shape[0]):
    if torontodf3['Neighbourhood'].iloc[i] == "Not assigned":
        torontodf3['Neighbourhood'].iloc[i] = torontodf3['Borough'].iloc[i]
        

In [40]:
torontodf3.shape

(103, 3)

### Read geospatial coordinates for postal codes

In [44]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
geo_data.shape

(103, 3)

### Rename first column of geospatial data frame so it matches the other data frame

In [47]:
geo_data.columns = ['Postcode', 'Latitude', 'Longitude']
geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the 2 data frames

In [50]:
torontodf4 = pd.merge(torontodf3, geo_data, on='Postcode')
torontodf4.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
